# Introduction To Recommender Systems

![title](resources/user_to_user.jpg)

# Goal of recommender systems

> *to predict the "rating" or "preference" a user would give to an (unseen) item*<br>

<center><img src="resources/Example.png" width="800"></center>

What rate do you guess that user 1 gives item 3?

##  Technology evolution of recommenders
<center><img src="resources/history.png" height="500"></center>

## Content-based recommendation
<center><img src="resources/contentbased.png" height="500"></center>

## Collaborative Filtering recommendation engine
<center><img src="resources/CF.png" width="800"></center>

##  Technology evolution of recommenders
<center><img src="resources/history.png" height="500"></center>

##  Collaborative Filtering Methods
- User-to-User Based: Users who are similar to you also liked ...
- Item-to-Item Based: Users who liked this item also liked... 


## User-to-User based
<center><img src="resources/user-based.png" height="700"></center>

## Item-to-Item based
<center><img src="resources/item-based.png" height="700"></center>

# Typical User-Item Matrix 
- The matrix contains either explicit (ratings) or implicit feedback (bought/clicked/viewed)
<img src="resources/BLOG_CCA_8.png" width="1000">


# User-to-User Simularity
<center><img src="resources/BLOG_CCA_11.png" width="600"></center>

# Item-to-Item Simularity

<center><img src="resources/BLOG_CCA_10.png" width="600"></center>

## Similarity metrics used in recommendation engines
* Cosine distance <br>
$
d^{cosine}(\mathbf{x}_a,\mathbf{x}_b)
 = \frac{ \mathbf{x}_a \cdot \mathbf{x}_b }{ \left \| \mathbf{x}_a \right \| \left \| \mathbf{x}_b \right \| }
 = \frac{ \sum_{i=1}^k x_{a,i}\cdot x_{b,i} }{\sqrt{\sum_{i=1}^k x_{a,i}^2 } \sqrt{\sum_{i=1}^k x_{b,i}^2} }
$
* Pearson correlation distance<br>
$
d^{pearson}(\mathbf{x}_a,\mathbf{x}_b)
 = \frac{ \sum_{i=1}^k (x_{a,i} - \bar{\mathbf{x}}_a) \cdot (x_{b,i} - \bar{\mathbf{x}}_b) }{\sqrt{\sum_{i=1}^k (x_{a,i} - \bar{\mathbf{x}}_a)^2 } \sqrt{\sum_{i=1}^k (x_{b,i} - \bar{\mathbf{x}}_b)^2} }
$
* Euclidean distance
* Manhattan distance
* Jaccard distance


# Hands on a transaction dataset

## Read in the transactional data

In [1]:
import numpy as np, pandas as pd
transactions = pd.read_pickle('data/transactions.pkl')

n_users = transactions.user_id.unique().shape[0]
n_items = transactions.product_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of products = ' + str(n_items))
print(f'Number of rows in transactional data = {transactions.shape[0]}')

Number of users = 5000 | Number of products = 1000
Number of rows in transactional data = 453836


In [2]:
transactions.sort_values(by=['user_id','order_id','product_id'],inplace=True)
transactions = transactions[['user_id','order_id','product_id','product_name']]
transactions.head(20)

,user_id,order_id,product_id,product_name
4698597,18,1020460,21019,Roasted Red Pepper Hummus
2762349,18,1020460,21137,Organic Strawberries
10491102,18,1020460,24830,Organic Romaine
8142399,18,1020460,26209,Limes
11926926,18,1020460,34126,Organic Italian Parsley Bunch
19160225,18,1020460,38739,Hass Avocado
15174494,18,2461523,5450,Small Hass Avocado
4483330,18,2461523,8518,Organic Red Onion
587782,18,2461523,17461,Air Chilled Organic Boneless Skinless Chicken ...
2955579,18,2461523,21137,Organic Strawberries


<center><img src="resources/simplify.jpg" width="600"></center>

### During this workshop, we do not look at frequency. Only, whether a user bought an item or not.

In [3]:
transactions = transactions.drop('order_id',axis=1).drop_duplicates() 
# only keep single interactions

In [4]:
transactions.sort_values(by='user_id').head()

,user_id,product_id,product_name
4698597,18,21019,Roasted Red Pepper Hummus
2402823,18,31717,Organic Cilantro
7098624,18,5876,Organic Lemon
20103043,18,4461,Organic Raw Unfiltered Apple Cider Vinegar
17431079,18,22888,90% Lean Ground Beef


# Normalize user_id
As we go to a sparse representation of the matrix later, we make the id's of both users and products range from 0 to n_users and n_items respectively. 

In [5]:
from resources.tools import Normalizer
user_normalizer = Normalizer(transactions.user_id.unique())
transactions['user'] = user_normalizer.apply(transactions.user_id)

item_normalizer = Normalizer(transactions.product_id.unique())
transactions['item'] = item_normalizer.apply(transactions.product_id)

In [6]:
transactions.sort_values(by=['user','item'],inplace=True)
transactions.head(50)

,user_id,product_id,product_name,user,item
20103043,18,4461,Organic Raw Unfiltered Apple Cider Vinegar,0,72
15174494,18,5450,Small Hass Avocado,0,101
7098624,18,5876,Organic Lemon,0,115
4483330,18,8518,Organic Red Onion,0,167
587782,18,17461,Air Chilled Organic Boneless Skinless Chicken ...,0,343
4698597,18,21019,Roasted Red Pepper Hummus,0,424
2762349,18,21137,Organic Strawberries,0,425
17431079,18,22888,90% Lean Ground Beef,0,461
10491102,18,24830,Organic Romaine,0,500
8142399,18,26209,Limes,0,532


# Split the ratings for train and test

In [7]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(transactions, test_size=0.2,random_state=3)

# Create user-item matrix from set of ratings

In [8]:
# We fill in the user item matrix for train and test set. 
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line.user, line.item] = 1

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line.user, line.item] = 1

# Determine how sparse/dense the train matrix is

In [9]:
# your code:


# Calculate cosine simularity matrices from the train set
$
s_u^{cos}(i_m,i_b)
 = \frac{ i_m \cdot i_b }{ \left \| i_m \right \| \left \| i_b \right \| }
 = \frac{ \sum x_{a,m} x_{a,b} }{ \sqrt{ \sum x_{a,m}^2 \sum x_{a,b}^2 } }
$

$
s_u^{cos}(u_k,u_a)
 = \frac{ u_k \cdot u_a }{ \left \| u_k \right \| \left \| u_a \right \| }
 = \frac{ \sum x_{k,m}x_{a,m} }{ \sqrt{\sum x_{k,m}^2\sum x_{a,m}^2} }
$

Hint: checkout pairwise_distances from sklearn 

In [10]:
# your code:


# What 10 items are most similar to 'Organic Hass Avocado'?

<center><img src="resources/avocado.jpg" width="600"></center>

In [11]:
# your code:


# Which products has user 10370 bought?

In [12]:
# your code:


# Now it's time to predict using our item-to-item similarity matrix
- A prediction can be made by (dot) multiplying the simularity matrix (#Items x #Items) by someone's profile (1 x #Items) <br>

$
\pmatrix{1.0 & 0.2 & 0.5 & 0.1 \\ 
         0.2 & 1.0 & 0.9 & 0.3 \\
         0.5 & 0.9 & 1.0 & 0.3 \\
         0.1 & 0.3 & 0.3 & 1.0} \pmatrix{0 \\ 1 \\ 1 \\ 0}  = 
         \pmatrix{0.7 \\ 1.9 \\ 1.9 \\ 0.6} \rightarrow \pmatrix{ \#1 \\ \texttt{not recommended} \\ \texttt{not recommended} \\ \#2}
$

- Grab user 10370 from the train tranactions and predict 20 items to recommend to this user.

In [13]:
# your code:


# How well did you do?

In [14]:
# your code:


## Is this good?

# Metrics for recommendations

|||
|-----------------:|-------------|--|
| precision  | % of correct rec.    | dependent on k|
| recall    | % of correct from test| dependent on k|
| f1 score  | harmonic mean of precision and recall    | dependent on k |
| mean average precision | front loading correct recommendations  | less dependend on k | 
| mean reciprical rank  | harmonic mean of rank |less dependent on k | 
|||

Calculate precision, recall and f1 score for k = 20 for user 10370

In [15]:
# your code:


# Model-based recommender


* **Before**: Recommendations based on similarity.

* **Now**: We model the observed interactions as the product of item features $x^{(i)}$ and user features $\theta^{(u)}$, such that <br>
$(\theta^{(u)})^T x^{(i)} \approx y^{(u,i)} $<br>
holds for all items

<center><img src="resources/ALS.png" width="900"></center>

With the trained model, that is with the learned user and product features, we can then compute interactions for a given user and for unseen given products.

### But what features to use?

## Motivation: content based recommendation
<center><img src="resources/content.png" width="700"></center>

## motivation: content based recommendation
| Product \ User| 0| 1| 2| 3|$|$| $x_1$ (organic)  | $x_2$ (vitamins) | $x_3$ (proteine) |$|$|item_vector | 
|:--------      |--|--|--|--|-  |----              |----   |---|--|----------------------------------------------|
| Spaghetti     | 1    | 0  | 0     | 0    |$|$| 0.5 | 0.2 | 0.1 |$|$|$ i^{(0)}=\pmatrix{ 0.5, 0.2, 0.1}$
| ground beef   | 0    | 1  | 1     | 1    |$|$| 0.3 | 0.6 | 0.9 |$|$|$i^{(1)}=\pmatrix{ 0.3, 0.6, 0.9}$
| tomato        | 1    | 1  | 0     | 1    |$|$| 1   | 0.7 | 0.3 |$|$|$i^{(2)}=\pmatrix{ 1.0, 0.7, 0.3}$
| Bell pepper   | 1    | 1  | 1     | 1    |$|$| 1   | 0.9 | 0.4 |$|$|$i^{(3)}=\pmatrix{ 1.0, 0.9, 0.4}$
| White rice    | 0    | 1  | 1     | 0    |$|$| 0.5 | 0.1 | 0.1 |$|$|$i^{(4)}=\pmatrix{ 0.5, 0.1, 0.1}$

* Given $x^{(i)}$ we can optimize for user vectors, $\theta^{(u)}$, such that when multiplied with the item vectors, we best approximate the observed purchases $y^{(u,i)}$. 
* We can do this by solving:<br>
$
\underset{\theta^{(u)}}{\min{}}
\frac{1}{2} \sum_{j:r(i,j)=1} ((\theta^{(j)})^T x^{(i)} - y^{(i,j)})^2  + 
\frac{\lambda}{2} \sum_{k=1}^n (\theta_k^{(u)})^2
$

* However, are the features that we used optimal to explain the observed interactions?
* Subsequently, given user vectors $\theta^{(u)}$ optimize for item vectors $x^{(i)}$
* Alternating Least Squares: 
$\theta \rightarrow x \rightarrow \theta \rightarrow x  \rightarrow \theta \rightarrow x\rightarrow \dots$

* There exist many packages that implement this algorithm, for example `implicit`

# Neural network based models

* Recently, advanced deep learning frameworks
* More freedom with development
* Better training possibilities

## Youtube Recommender

<center><img src="resources/neuralnet.png" width="600"></center>

# Conclusion

* Recommendations everywhere
* Simple algorithms work well
* Trend to use implicit feedback only
* Active research area, especially focussed on deep learning techniques